![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [1]:
import pandas as pd

brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

In [2]:
merged_df=info.merge(finance,on="product_id",how="outer")
merged_df=merged_df.merge(reviews,on="product_id",how="outer")
merged_df=merged_df.merge(brands,on="product_id",how="outer")
merged_df.head()

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,brand
0,NaN,AH2430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,Adidas
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,Adidas
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,Adidas
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,Adidas


In [3]:
merged_df.dropna(inplace=True)
merged_df.head()

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,brand
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,Adidas
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,Adidas
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,Adidas
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,Adidas
5,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,47.99,19.20,0.6,1555.20,1.0,45.0,Adidas


What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?

In [4]:
merged_df["price_label"]=pd.qcut(merged_df["listing_price"],q=4,labels=["Budget","Average","Expensive","Elite"])

In [5]:
merged_df

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,brand,price_label
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,Adidas,Expensive
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,Adidas,Budget
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,Adidas,Expensive
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,Adidas,Expensive
5,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,47.99,19.20,0.6,1555.20,1.0,45.0,Adidas,Average
...,...,...,...,...,...,...,...,...,...,...,...
3174,Nike Tiempo Legend 8 Academy TF,AT6100-606,The Nike Tiempo Legend 8 Academy TF takes the ...,0.00,64.95,0.0,0.00,0.0,0.0,Nike,Budget
3175,Nike React Metcon AMP,CT9155-063,The Nike React Metcon AMP takes the stability ...,0.00,139.95,0.0,251.91,3.0,1.0,Nike,Budget
3176,Air Jordan 8 Retro,CI1236-100,The Air Jordan 8 Retro recaptures the memorabl...,159.95,127.97,0.0,230.35,5.0,1.0,Nike,Elite
3177,Nike Air Max 98,AH6799-300,The Nike Air Max 98 features the OG design lin...,0.00,169.95,0.0,1223.64,4.0,4.0,Nike,Budget


This function pd.qcut divides the values in the "listing_price" column into four equal-sized bins or quartiles (q=4). It assigns a label to each bin based on the provided labels list. The lowest values will be labeled as "Budget," the next quartile as "Average," the third quartile as "Expensive," and the highest quartile as "Elite."

In [5]:
adidas_vs_nike = merged_df.groupby(["brand", "price_label"], as_index=False).agg(
    num_products=("price_label", "count"), 
    mean_revenue=("revenue", "mean")
).round(2).reset_index(drop=True)

The parameter drop=True ensures that the previous index is not included 

In [7]:
adidas_vs_nike.head()

,brand,price_label,num_products,mean_revenue
0,Adidas,Budget,574,2015.68
1,Adidas,Average,655,3035.30
2,Adidas,Expensive,759,4621.56
3,Adidas,Elite,587,8302.78
4,Nike,Budget,357,1596.33


What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?

In [6]:
merged_df["description_length"]=merged_df["description"].str.len()
merged_df["description_length"]

1       175
2       172
3       264
4       288
5       221
       ... 
3174    146
3175    378
3176    204
3177    240
3178    202
Name: description_length, Length: 3117, dtype: int64

In [7]:
max(merged_df["description_length"])

687

In [8]:
# Upper description length limits
lengthes = [0, 100, 200, 300, 400, 500, 600, 700]

In [9]:
labels = ["100", "200", "300", "400", "500", "600", "700"]

In [10]:
merged_df["description_length"] = pd.cut(merged_df["description_length"], bins=lengthes, labels=labels)

In [11]:
merged_df.head()

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,brand,price_label,description_length
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,Adidas,Expensive,200
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,Adidas,Budget,200
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,Adidas,Expensive,300
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,Adidas,Expensive,300
5,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,47.99,19.20,0.6,1555.20,1.0,45.0,Adidas,Average,300


In [12]:
description_length=merged_df.groupby("description_length",as_index=False).agg(mean_rating=("rating","mean"),num_reviews=("reviews","count")).round(2)

In [21]:
description_length

,description_length,mean_rating,num_reviews
0,100,2.26,7
1,200,3.19,526
2,300,3.28,1785
3,400,3.29,651
4,500,3.35,118
5,600,3.12,15
6,700,3.65,15


In [13]:
mylist = "shoe*|trainer*|foot*"

In [14]:
shoes = merged_df[merged_df["description"].str.contains(mylist)]

In [15]:
shoes

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,brand,price_label,description_length
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,Adidas,Expensive,200
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,Adidas,Budget,200
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,Adidas,Expensive,300
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,Adidas,Expensive,300
5,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,47.99,19.20,0.6,1555.20,1.0,45.0,Adidas,Average,300
...,...,...,...,...,...,...,...,...,...,...,...,...
3168,Nike Zoom Pegasus Turbo XX,AR4347-600,"Bridging performance and style, the Nike Zoom ...",169.95,135.97,0.0,244.75,1.0,1.0,Nike,Elite,500
3169,Nike Air Max 200,CI3867-001,Inspired by energy patterns radiating from Ear...,0.00,99.95,0.0,0.00,0.0,0.0,Nike,Budget,300
3171,Nike Mercurial Vapor 13 Club MG,AT7968-606,The Nike Mercurial Vapor 13 Club MG wraps your...,0.00,49.95,0.0,0.00,0.0,0.0,Nike,Budget,200
3173,Air Jordan 5 Retro,CD2722-001,The Air Jordan 5 Retro for women gives a colou...,0.00,159.95,0.0,863.73,3.3,3.0,Nike,Budget,300


In [16]:
clothing = merged_df[~merged_df.isin(shoes["product_id"])]

In [17]:
clothing

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,brand,price_label,description_length
1,Women's adidas Originals Sleek Shoes,NaN,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,Adidas,Expensive,200
2,Women's adidas Swim Puka Slippers,NaN,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,Adidas,Budget,200
3,Women's adidas Sport Inspired Questar Ride Shoes,NaN,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,Adidas,Expensive,300
4,Women's adidas Originals Taekwondo Shoes,NaN,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,Adidas,Expensive,300
5,Women's adidas Sport Inspired Duramo Lite 2.0 ...,NaN,Refine your interval training in these women's...,47.99,19.20,0.6,1555.20,1.0,45.0,Adidas,Average,300
...,...,...,...,...,...,...,...,...,...,...,...,...
3174,Nike Tiempo Legend 8 Academy TF,AT6100-606,The Nike Tiempo Legend 8 Academy TF takes the ...,0.00,64.95,0.0,0.00,0.0,0.0,Nike,Budget,200
3175,Nike React Metcon AMP,NaN,The Nike React Metcon AMP takes the stability ...,0.00,139.95,0.0,251.91,3.0,1.0,Nike,Budget,400
3176,Air Jordan 8 Retro,CI1236-100,The Air Jordan 8 Retro recaptures the memorabl...,159.95,127.97,0.0,230.35,5.0,1.0,Nike,Elite,300
3177,Nike Air Max 98,AH6799-300,The Nike Air Max 98 features the OG design lin...,0.00,169.95,0.0,1223.64,4.0,4.0,Nike,Budget,300


In [19]:
clothing.dropna(inplace=True)
clothing

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,brand,price_label,description_length
71,Men's adidas Toe Side II Slippers,CM0068,The adidas casual sandals for men with a strik...,12.99,7.79,0.4,827.30,1.5,59.0,Adidas,Budget,300
100,Men's adidas Running Nepton 2.0 Shoes,CL7470,Energize your run while staying comfortable wi...,55.99,27.99,0.5,251.91,4.0,5.0,Adidas,Average,300
105,Men's adidas Toe Side II Slippers,CM0069,The adidas casual sandals for men with a strik...,12.99,7.79,0.4,308.48,3.0,22.0,Adidas,Budget,300
114,Men's adidas Running Nepton 2.0 Shoes,CL7471,Energize your run while staying comfortable wi...,55.99,27.99,0.5,1813.75,3.8,36.0,Adidas,Average,300
136,Women's Originals Sleek Super Shoes,EF4954,These patent leather adidas Sleek Super Shoes ...,79.99,55.99,0.3,9675.07,2.2,96.0,Adidas,Expensive,200
...,...,...,...,...,...,...,...,...,...,...,...,...
3172,Nike Zoom Pegasus Turbo 2 Special Edition,BV7757-500,The Nike Zoom Pegasus Turbo 2 is updated with ...,159.95,127.97,0.0,230.35,5.0,1.0,Nike,Elite,200
3174,Nike Tiempo Legend 8 Academy TF,AT6100-606,The Nike Tiempo Legend 8 Academy TF takes the ...,0.00,64.95,0.0,0.00,0.0,0.0,Nike,Budget,200
3176,Air Jordan 8 Retro,CI1236-100,The Air Jordan 8 Retro recaptures the memorabl...,159.95,127.97,0.0,230.35,5.0,1.0,Nike,Elite,300
3177,Nike Air Max 98,AH6799-300,The Nike Air Max 98 features the OG design lin...,0.00,169.95,0.0,1223.64,4.0,4.0,Nike,Budget,300


In [21]:
product_types = pd.DataFrame({"num_clothing_products": len(clothing), 
                        "median_clothing_revenue":clothing["revenue"].median(), 
                              "num_footwear_products": len(shoes), 
                        "median_footwear_revenue":shoes["revenue"].median()}, 
                              index=[0])

In [22]:
product_types

,num_clothing_products,median_clothing_revenue,num_footwear_products,median_footwear_revenue
0,478,625.07,2639,3073.3
